In [ ]:
import os
import gc
import logging
import time
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve, accuracy_score, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
#from sklearn import svm
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
#from sklearn import cluster

from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, BatchNormalization, Input, Conv2D
from keras import callbacks
from keras import metrics
from keras.optimizers import Adam
from keras import backend as K
import keras
from keras.models import Model, Sequential
from keras.models import model_from_json
from keras import regularizers
from keras.losses import binary_crossentropy
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasClassifier

import tensorflow as tf

In [ ]:
%matplotlib inline

In [ ]:
batch_size = 100

In [ ]:
def auc(y_true, y_pred): 
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

In [ ]:
def step_decay(epoch):
    initial_lrate = 0.1
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    return lrate

In [ ]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []
 
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(step_decay(len(self.losses)))

In [ ]:
def simply_blend(prediction_dataframes, weights, target_column_name='target'):
    blended_prediction_df = pd.DataFrame(data=np.zeros(prediction_dataframes[0].shape[0]), columns=[target_column_name])
    for prediction_df, weight in zip(prediction_dataframes, weights):
        blended_prediction_df[target_column_name] = blended_prediction_df[target_column_name] + weight * prediction_df[target_column_name]
        #blended_prediction_df.add(weight * prediction_df)
    return blended_prediction_df / sum(weights)

In [ ]:
def train_keras_nn(train_features, train_targets, model, batch_size=100, epochs=20, n_splits=5):
    loss_history = LossHistory()
    lrate = LearningRateScheduler(step_decay)
    callbacks_list = [EarlyStopping(monitor='val_auc', patience=20, mode='max'), loss_history, annealer]
    sss = StratifiedShuffleSplit(n_splits=n_splits)
    start_time = time.time()
    for train_index, test_index in sss.split(train_features, train_targets):
        X_train, X_val = train_features[train_index], train_features[test_index]
        Y_train, Y_val = train_targets[train_index], train_targets[test_index]
        #X_tr, Y_tr = augment(X_train, Y_train)
        #print("{} iteration".format(i+1))
        history= model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, callbacks=callbacks_list, verbose=1, validation_data=(X_val,Y_val))
        #history= sequential_nn_model.fit(X_train, Y_train, batch_size=batch_size, epochs=50, callbacks=callbacks_list, verbose=1, validation_data=(X_val,Y_val))
        del X_train, X_val, Y_train, Y_val
        gc.collect()
    print("Run time {} min".format((time.time() - start_time) / 60))
    return model

In [ ]:
def train_lgbm(train_df, test_df, target, features, param, num_round=1000000):
    start_time = time.time()
    folds = StratifiedKFold(n_splits=10, shuffle=False, random_state=44000)
    oof = np.zeros(len(train_df))
    predictions = np.zeros(len(test_df))
    feature_importance_df = pd.DataFrame()
    lgb_classifier = None
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
        print("Fold {}".format(fold_))
        trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
        val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])
        num_round = num_round
        clf = lgb.train(
            param,
            trn_data,
            num_round,
            valid_sets=[trn_data, val_data],
            verbose_eval=1000,
            early_stopping_rounds=3000
        )
        lgb_classifier = clf
        oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
        fold_importance_df = pd.DataFrame()
        fold_importance_df['Feature'] = features
        fold_importance_df['importance'] = clf.feature_importance()
        fold_importance_df['fold'] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        predictions += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits
    print("Total run time {} min:".format((time.time() - start_time) / 60))
    print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))
    return oof, predictions, feature_importance_df, clf

In [ ]:
''''
kernel_regularizer=regularizers.l2(0.01)
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(train_polinomial_values_ucm_1_2.shape[1],)))
#model.add(PreLU(alpha=.001))
model.add(Dropout(0.6))
model.add(BatchNormalization())
model.add(Dense(64, input_shape=(train_polinomial_values_ucm_1_2.shape[1] / 2, ), activation='relu'))
#model.add(PreLU(alpha=.001))
model.add(Dropout(0.6))
model.add(BatchNormalization())
model.add(Dense(32, input_shape=(train_polinomial_values_ucm_1_2.shape[1] / 4, ), activation='relu'))
#model.add(PreLU(alpha=.001))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))

annealer = LearningRateScheduler(lambda x: 1e-2 * 0.95 ** x)
'''

In [ ]:
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', auc])

In [ ]:
'''
sequential_nn_model = Sequential()
sequential_nn_model.add(Dense(batch_size, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
sequential_nn_model.add(Dropout(0.1))
#sequential_nn_model.add(Dropout(0.4))
sequential_nn_model.add(BatchNormalization())
sequential_nn_model.add(Dense(batch_size, input_dim=input_dim / 2, kernel_initializer='normal', activation='sigmoid'))
####sequential_nn_model.add(Dense(batch_size, input_dim=input_dim / 10, kernel_initializer='normal', activation='sigmoid'))
sequential_nn_model.add(Dropout(0.1))
####sequential_nn_model.add(Dropout(0.4))
sequential_nn_model.add(BatchNormalization())
sequential_nn_model.add(Dense(batch_size, input_dim=input_dim / 4, kernel_initializer='normal', activation='relu'))
sequential_nn_model.add(Dropout(0.1))
sequential_nn_model.add(BatchNormalization())
sequential_nn_model.add(Dense(batch_size, input_dim=input_dim / 4, kernel_initializer='normal', activation='sigmoid'))
#sequential_nn_model.add(Dense(batch_size, input_shape=(100, 200), kernel_initializer='normal', activation='sigmoid'))
#sequential_nn_model.add(Dropout(0.76))
#sequential_nn_model.add(Dropout(0.24))
sequential_nn_model.add(Dropout(0.1))
sequential_nn_model.add(BatchNormalization())
sequential_nn_model.add(Dense(batch_size, input_dim=input_dim / 30, kernel_initializer='normal', activation='relu'))
sequential_nn_model.add(Dropout(0.1))
sequential_nn_model.add(BatchNormalization())
sequential_nn_model.add(Dense(batch_size, input_dim=input_dim / 30, kernel_initializer='normal', activation='sigmoid'))
sequential_nn_model.add(Dropout(0.1))
sequential_nn_model.add(BatchNormalization())
sequential_nn_model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
'''

In [ ]:
#sequential_nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', auc])

In [ ]:
'''
def make_sequential_model_min(input_dim):
    sequential_nn_model_min = Sequential()
    sequential_nn_model_min.add(Dense(batch_size, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
    sequential_nn_model_min.add(Dropout(0.4))
    sequential_nn_model_min.add(BatchNormalization())
    sequential_nn_model_min.add(Dense(batch_size, input_dim=input_dim / 10, kernel_initializer='normal', activation='sigmoid'))
    sequential_nn_model_min.add(Dropout(0.4))
    sequential_nn_model_min.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    return sequential_nn_model_min
'''

In [ ]:
#sequential_nn_model_min = make_sequential_model_min(train_polinomial_values_ucm_1_2.shape[1])

In [ ]:
#sequential_nn_model_min.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', auc])

In [ ]:
train_df = pd.read_csv('../input/train.csv')

In [ ]:
test_df = pd.read_csv('../input/test.csv')

In [ ]:
train_df.info()

In [ ]:
train_df_id_droped = train_df[train_df.columns.drop('ID_code')]

In [ ]:
uniques_dict = {column_name: train_df_id_droped[column_name].unique() for column_name in train_df_id_droped.columns.drop('target').tolist()}

In [ ]:
uniques_dict_counts = {column_name: uniques.shape[0] for column_name, uniques in uniques_dict.items()}

In [ ]:
uniques_counts_series = pd.Series(uniques_dict_counts)

In [ ]:
uniques_counts_series[:5]

In [ ]:
uniques_counts_series.unique().shape

In [ ]:
uniques_counts_series.max()

In [ ]:
uniques_counts_series.min()

In [ ]:
train_df_id_droped.shape

In [ ]:
features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
target = train_df['target']

In [ ]:
param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.4,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.05,
    'learning_rate': 0.01,
    'max_depth': -1,  
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': 1
}

In [ ]:
'''
cols = (feature_importance_df[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:150].index)
best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

plt.figure(figsize=(14,28))
sns.barplot(x="importance", y="Feature", data=best_features.sort_values(by="importance",ascending=False))
plt.title('Features importance (averaged/folds)')
plt.tight_layout()
plt.savefig('FI.png')
'''

In [ ]:
min_max_scaler = MinMaxScaler()

In [ ]:
#train_df_rows_count = train_df.shape[0]

In [ ]:
#train_df_rows_count

In [ ]:
#uniques_count_more_1_2 = uniques_counts_series[uniques_counts_series > train_df_rows_count / 2]

In [ ]:
#uniques_count_more_1_2.shape

In [ ]:
#uniques_count_more_1_4_less_1_2 = uniques_counts_series[uniques_counts_series < train_df_rows_count / 2]

In [ ]:
#uniques_count_less_1_2_more_1_4 = uniques_count_more_1_4_less_1_2[uniques_count_more_1_4_less_1_2 > train_df_rows_count / 4]

In [ ]:
#uniques_count_more_1_4_less_1_2.shape

In [ ]:
#uniques_count_less_1_4 = uniques_counts_series[uniques_counts_series < train_df_rows_count / 4]

In [ ]:
#uniques_count_less_1_4.shape

In [ ]:
target_values = train_df['target'].values

In [ ]:
'''
train_results_whole = train_lgbm(
    train_df,
    test_df,
    train_df['target'],
    train_df.columns.drop(['ID_code', 'target']).tolist(),
    param
)
'''

In [ ]:
#oof_ucm_whole, predictions_whole, feature_importance_whole, clf_ucm_whole = train_results_whole

In [ ]:
#predictions_df_whole = pd.DataFrame(data=predictions_whole, columns=['target'])

In [ ]:
#submission_whole_only_df = pd.DataFrame({'ID_code': ID_code, 'target': predictions_df_whole['target'].values.astype('float32')})

In [ ]:
#submission_whole_only_df.to_csv('submission_whole_only.csv', index=False)

In [ ]:
#secquential_nn_model_min = train_keras_nn(train_polinomial_values_ucm_1_2, train_target_values_ucm_1_2, sequential_nn_model_min_1_2, batch_size=512, epochs=30)

In [ ]:
#loss_and_metrics = sequential_nn_model_min.evaluate(holdout_test_polinomial_values_ucm_1_2, holdout_test_target_values_ucm_1_2, batch_size=100)

In [ ]:
#sequential_nn_model_min.save('secquential_nn_model_min.txt')